<h1>Applied Data Science Capstone Project</h1>

<p>This notebook was put together for the IBM Data Science certificate capstone project. This first version will describe the business problem of the capstone and related data to solve problem.</p>

<h3>The business idea</h3>
<p>I have a friend who is considering on moving to the Capital area in Finland who has never lived in the area and is unaware of the neighbourhoods. Coming from rural areas he's a mainly concerned about about finding a location to live where: a) The pricing is not too expensive b) the available public transport to his new workplace in Otaniemi would be relative fast (45 - 60min max by public transport) c) the place is not too restless and unsafe d) there would be parks, gardens, cafeterias etc for unwinding on free time.</p>

<p>As the capital area is relatively large there are a lot of different neighbourhoods to choose from. There are certain "bad" neighbourhoods where the crimes are more common. These neighbourhoods usually also are either located in the transit hubs or have more bars etc in the areas. The speed of the public transportation also varies quite a bit depending on how many transit changes are required and how close the start and end points are to the main hubs. While the prices in the whole capital area are quite high in relation to other areas there is also a lot of difference across the neighbourhoods. The more quiet areas with more parks and less density of population are usually also quite far in terms of public transport.</p>

<p>It would be interesting to have a service that would recommend neighbourhoods based on this criteria when planning to purchase an apartment. Personally I spent around 8 months hunting for an apartment and trying to figure out the neighbourhoods. </p>

<p>Since the easiest way to find locations for neighbourhoods in FourSquare and other data sources includes postal codes we will mainly focus on different postal code areas. The actual code areas can include several neighbourhoods inside the area but this will be the precision we're aiming for in this project.</p>

<h3>The data</h3>
<p>Solving this problem requires quite a few separate services to obtain the data. The main types of data that we will need to solve the problem are: location data with different venues, data of the crime rates, dataset or a service to build the data set from the distances and estimated travel, average prices data for the neighbourhoods.</p>

<p><b>Location data</b> will come from FourSquare the data will be used to evaluate the "restlessness" or how lively the neighbourhood is and to obtain the most common types of venues and trying to make some recommendations based on the most desired venue types.</p>

<p>Since getting it from a primary source per neighbourhoods is quite hard <b>the crime data</b> will come from infogram that has been published in a newspaper and been built from data coming from Statistics Finland and the police. The data will be used to visualise the crime occurances on a map and as one basis for evaluating the optimum neighbourhood.</p>

<p><b>The average prices data</b> will be acquired from Statistics Finland and the obtained csv will have to be wrangled a bit to produce the average price of the apartments. The data will be used for ranking the neighbourhoods.</p>

<p><b>The public transport data</b> will be acquired from HSL which is the public transport provider for the whole capital area. The collected data will be used to rank the neighbourhoods based on the speed of the transport and to restrict neighbourhoods that are too far from the home to work transit time requirements.</p>

<p>To visualise the mentioned data on the map we will also need to obtain the geojson borders of the neighbourhoods in the areas. We can obtain the data from HSY Web Features service at <a href="https://kartta.hsy.fi/geoserver/wfs">https://kartta.hsy.fi/geoserver/wfs.</a></p>


First lets import all the dependencies.

In [192]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import folium
import requests
from sklearn.cluster import KMeans
import json

import matplotlib.cm as cm
import matplotlib.colors as colors

import geopandas as gp

import pyproj
from pyproj import CRS

from python_graphql_client import GraphqlClient

<b>1. The locations</b>

To begin solving the problem lets print out the map of the Capital to visualise the area we are discussing about. We can use geopys Notinatim API to to first acquire the location of the mainarea and the workplace and the we can use Folium to print the area on map and mark the workplace there.

In [145]:
# Use a neighbourhood a bit north of the actual city center to center the map on more on the mainland
address_kannelmaki = 'Kannelmäki, Helsinki, Finland'

geolocator = Nominatim(user_agent="helsinki_explorer")
location_kannelmaki = geolocator.geocode(address_kannelmaki)
latitude_kannelmaki = location_kannelmaki.latitude
longitude_kannelmaki = location_kannelmaki.longitude
print('The geograpical coordinates of {} is latitude: {} and longitude: {}.'.format(address_kannelmaki, latitude_kannelmaki, longitude_kannelmaki))

# Use a neighbourhood a bit north of the actual city center to center the map on more on the mainland
address_workplace = 'Innopoli 3, Espoo, Finland'

geolocator = Nominatim(user_agent="helsinki_explorer")
location_workplace = geolocator.geocode(address_workplace)
latitude_workplace = location_workplace.latitude
longitude_workplace = location_workplace.longitude
print('The geograpical coordinates of {} is latitude: {} and longitude: {}.'.format(address_workplace, latitude_workplace, longitude_workplace))


The geograpical coordinates of Kannelmäki, Helsinki, Finland is latitude: 60.2436076 and longitude: 24.8832893.
The geograpical coordinates of Innopoli 3, Espoo, Finland is latitude: 60.1881158 and longitude: 24.80870540096568.


In [148]:
# Generate at new folium map from latitude and longitude values
map_uusimaa = folium.Map(location=[latitude_kannelmaki, longitude_kannelmaki], zoom_start=10)

# Mark the workplace on the map
folium.CircleMarker(
        [latitude_workplace, longitude_workplace],
        radius=5,
        popup='Workplace',
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_uusimaa) 

map_uusimaa

<b>2. Postal code areas, the neighbourhoods and the corresponding geojson</b>

Next lets load the geojson for neighbourhoods in the capital region. The geojson has been attained from HSY's web features service and converted into geojson. The data has been included in the repository so we can just load it. We use geopandas to load the data. 

In [243]:
# Read the file with geopandas
neighbourhoods = gp.read_file('postinro_alue.geojson')

# Print out first five rows of the data for inspection
neighbourhoods.head()

,gml_id,posno,toimip,toimip_ru,nimi,nimi_ru,kunta,kunta_nro,geometry
0,pks_postinumeroalueet_2020.1,00100,HELSINKI,HELSINGFORS,Helsinki Keskusta - Etu-Töölö,Helsingfors centrum - Främre Tölö,Helsinki,091,"MULTIPOLYGON (((25495415.010 6673755.420, 2549..."
1,pks_postinumeroalueet_2020.2,00120,HELSINKI,HELSINGFORS,Punavuori,Rödbergen,Helsinki,091,"MULTIPOLYGON (((25496720.730 6672703.770, 2549..."
2,pks_postinumeroalueet_2020.3,00130,HELSINKI,HELSINGFORS,Kaartinkaupunki,Gardesstaden,Helsinki,091,"MULTIPOLYGON (((25496776.230 6672752.055, 2549..."
3,pks_postinumeroalueet_2020.4,00140,HELSINKI,HELSINGFORS,Kaivopuisto - Ullanlinna,Brunnsparken - Ulrikasborg,Helsinki,091,"MULTIPOLYGON (((25497132.180 6672015.420, 2549..."
4,pks_postinumeroalueet_2020.5,00150,HELSINKI,HELSINGFORS,Eira - Hernesaari,Eira - Ärtholmen,Helsinki,091,"MULTIPOLYGON (((25496970.120 6671136.315, 2549..."


From the geometry we can also see that the data is in EPSG:3879 format. To use the geometries in Folium we need to project the geometries to EPSG:4326.

In [244]:
# Define the crs of the dataframe
neighbourhoods.crs = CRS.from_epsg(3879)

# Project to EPSG:4326
neighbourhoods = neighbourhoods.to_crs(epsg=4326)

# Print the first five rows for inspection
neighbourhoods.head()

,gml_id,posno,toimip,toimip_ru,nimi,nimi_ru,kunta,kunta_nro,geometry
0,pks_postinumeroalueet_2020.1,00100,HELSINKI,HELSINGFORS,Helsinki Keskusta - Etu-Töölö,Helsingfors centrum - Främre Tölö,Helsinki,091,"MULTIPOLYGON (((24.91739 60.17664, 24.91766 60..."
1,pks_postinumeroalueet_2020.2,00120,HELSINKI,HELSINGFORS,Punavuori,Rödbergen,Helsinki,091,"MULTIPOLYGON (((24.94093 60.16721, 24.94107 60..."
2,pks_postinumeroalueet_2020.3,00130,HELSINKI,HELSINGFORS,Kaartinkaupunki,Gardesstaden,Helsinki,091,"MULTIPOLYGON (((24.94193 60.16764, 24.95107 60..."
3,pks_postinumeroalueet_2020.4,00140,HELSINKI,HELSINGFORS,Kaivopuisto - Ullanlinna,Brunnsparken - Ulrikasborg,Helsinki,091,"MULTIPOLYGON (((24.94835 60.16103, 24.94846 60..."
4,pks_postinumeroalueet_2020.5,00150,HELSINKI,HELSINGFORS,Eira - Hernesaari,Eira - Ärtholmen,Helsinki,091,"MULTIPOLYGON (((24.94545 60.15314, 24.94237 60..."


Lets format the data frame a bit and also translate the names into english.

In [245]:
# We can drop some columns we don't need
neighbourhoods = neighbourhoods.drop(labels=['toimip_ru', 'nimi_ru', 'gml_id', 'kunta_nro', 'toimip'], axis = 1)

# Lets also rename and translate the columns
neighbourhoods.columns = ['PostalCode', 'Neighbourhood', 'Municipality', 'Geometry']

# Lets print out the dataframe to see the applied changes
neighbourhoods.head()

,PostalCode,Neighbourhood,Municipality,Geometry
0,00100,Helsinki Keskusta - Etu-Töölö,Helsinki,"MULTIPOLYGON (((24.91739 60.17664, 24.91766 60..."
1,00120,Punavuori,Helsinki,"MULTIPOLYGON (((24.94093 60.16721, 24.94107 60..."
2,00130,Kaartinkaupunki,Helsinki,"MULTIPOLYGON (((24.94193 60.16764, 24.95107 60..."
3,00140,Kaivopuisto - Ullanlinna,Helsinki,"MULTIPOLYGON (((24.94835 60.16103, 24.94846 60..."
4,00150,Eira - Hernesaari,Helsinki,"MULTIPOLYGON (((24.94545 60.15314, 24.94237 60..."


Examine the dimensions of the dataframe.

In [246]:
neighbourhoods.shape

(172, 4)

<b>3. Transit data via HSL</b>

To calculate the distances to different postal code areas we need to access the HSL GraphQL API. The documentation for the API is available <a href="https://digitransit.fi/en/developers/apis/1-routing-api/x-advanced/">here</a>. We will use python_graphql_client library for querying the API. The postal code areas can be somewhat large and differ between the actual apartment destination, but we will just cut some corners and estimate based on whatever coordinates the Nominatim library provides for the postal code.

Lets first setup the client and the query for loading the transit data. Since we are estimating the time to workplace then lets assume that we want to get there around the rush hour. Lets plan the routes to start on monday at eight in the morning.

In [236]:
# Setup endpoint to Helsinki area graphql api
endpoint = "https://api.digitransit.fi/routing/v1/routers/hsl/index/graphql"
# Setup the client
client = GraphqlClient(endpoint=endpoint)

# Define the query
query = """
    query planQuery($start: InputCoordinates, $end: InputCoordinates) {
        plan(
            from: $start
            to: $end
            date: "2021-06-07"
            time: "08:00:00"
        ) {
            itineraries {
                duration
            }
        }
    }
"""

Then lets define some test location and use it in the query to validate that the query is working and to check how the data looks.

In [259]:
# Get the location for the first postcode in the dataframe
address_hsy_test = '00100, Finland'

geolocator = Nominatim(user_agent="helsinki_explorer")
location_hsy_test = geolocator.geocode(address_hsy_test)
latitude_hsy_test = location_hsy_test.latitude
longitude_hsy_test = location_hsy_test.longitude

# Setup some test variables
variables = { "start": {"lat": latitude_hsy_test, "lon": longitude_hsy_test}, "end": {"lat": latitude_workplace, "lon": longitude_workplace}}

# Execute the query
data = client.execute(query=query, variables=variables)

routes = data['data']['plan']['itineraries'];

# Print the data for inspection
print(routes)

[{'duration': 1979}, {'duration': 1979}, {'duration': 1919}]


Looking at the data we can see that for the plan it returns several different route options with varying durations defined in seconds.

In [266]:
# Check the shortest duration, floor to minutes
min(map(lambda route: route['duration'], routes)) // 60

31

<p>
Now lets form the durations dataframe. We will take the postal codes from the neighbourhoods dataframe and use it for looping over the postal codes. Then we will get the postal code using Nominatim and then use the location for querying the data from HSY and append the postal code, lon, lat and the duration into a dataframe.
</p>
<p>
Since the loading will take a long while we will also later save the data into a csv for safety. Also we are somewhat in the mercy of the HSY API so will check the routes that cannot be found and just ignore those postal codes for this project. To avoid the routes not being found the position of the geospatial coordinates could be more intelligent and not just a "guess".
</p>


In [270]:
# Create new dataframe for durations data
durations = pd.DataFrame(columns=['PostalCode', 'Duration', 'Longitude', 'Latitude'])

# Create a dict from the data for looping
codes = neighbourhoods['PostalCode']

for code in codes:
    # Get the geospatial coordinates for the postal code
    address = code + ', Finland'
    geolocator = Nominatim(user_agent="helsinki_explorer")
    hsy_location = geolocator.geocode(address)
    hsy_latitude = hsy_location.latitude
    hsy_longitude = hsy_location.longitude
    
    # Setup variables for the query
    variables = { "start": {"lat": hsy_latitude, "lon": hsy_longitude}, "end": {"lat": latitude_workplace, "lon": longitude_workplace}}

    # Execute the query
    data = client.execute(query=query, variables=variables)
    
    # Extract the itineraries
    routes = data['data']['plan']['itineraries']
    if (len(routes) == 0):
        print('Unable to find routes for code', code)
        continue
        
    # Calculate the minimum duration and convert to minutes
    duration = min(map(lambda route: route['duration'], routes)) // 60
    
    # Append to dataframe
    durations = durations.append(
        { 'PostalCode': code, 'Duration': duration, 'Latitude': hsy_latitude, 'Longitude': hsy_longitude },
        ignore_index=True)
    
# Inspect the data
durations.head()

Unable to find routes for code 00310


Unable to find routes for code 01800
Unable to find routes for code 02290


Unable to find routes for code 02980


,PostalCode,Duration,Longitude,Latitude
0,00100,31,24.933727,60.169989
1,00120,42,24.939202,60.163562
2,00130,44,24.947547,60.165009
3,00140,52,24.952425,60.158122
4,00150,44,24.938014,60.158939


In [272]:
# Add the dataframe for safety, in case the queries fail you could just load the data from the repo
# durations.to_csv('postinro_durations.csv', index=False)
# durations = pd.read_csv('postinro_durations.csv')

Seems like the API could not find routes for a few of the routes. Lets investigate a bit. To see how much data we got in comparison to the original post codes data. We will ignore the postal codes for this project.

In [274]:
# Check the obtained durations data dimensions
print(durations.shape)
# Check the neighbourhoods data dimensions
print(neighbourhoods.shape)

(168, 4)
(172, 4)


<b>4. Average prices data</b>

<b>5. The crime data</b>

<b>6. Venues data from FourSquare</b>

In [142]:
with open('postinro_alue.geojson') as f:
  geojson = json.load(f)

# Lets print the first feature in the geojson for inspection
geojson['features'][0]

{'type': 'Feature',
 'properties': {'gml_id': 'pks_postinumeroalueet_2020.1',
  'posno': '00100',
  'toimip': 'HELSINKI',
  'toimip_ru': 'HELSINGFORS',
  'nimi': 'Helsinki Keskusta - Etu-Töölö',
  'nimi_ru': 'Helsingfors centrum - Främre Tölö',
  'kunta': 'Helsinki',
  'kunta_nro': '091'},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[25495415.01008449, 6673755.42],
     [25495429.86006965, 6673757.805],
     [25495451.88004763, 6673670.115],
     [25495604.66989484, 6673712.91],
     [25495684.28981522, 6673744.305],
     [25495689.86980963, 6673802.16],
     [25495704.86979464, 6673801.98],
     [25495858.05014146, 6673836.135],
     [25495952.70004681, 6673857.24],
     [25496001.95999754, 6673868.22],
     [25496069.27993023, 6673882.815],
     [25496100.95989855, 6673906.83],
     [25496230.64976885, 6674007.87],
     [25496393.64010586, 6674134.17],
     [25496865.33013417, 6674098.17],
     [25496861.04013846, 6673876.17],
     [25496896.83010267, 6673744.275],
    

You can read more about geojson on <a href="https://en.wikipedia.org/wiki/GeoJSON">wikipedia</a> but basically the geojson is just standard for representing geographical features. The geojson can be of several types like points, polygons, lines etc. For this project we are interested in using geojson polygons to render the boundaries of each postal area.

The geojson can contain some extra properties, f.ex here it contains the neighbourhood names, the postal codes etc. The geojson also the type of the geometry and the corresponding coordinates.

Next lets populate a new pandas dataframe using the geojson properties.

In [150]:
neighbourhoods_data = 

# Lets loop over the feature and populate the dataframe
for feature in geojson['features']:
    postal_code = feature['properties']['posno']
    municipality = feature['properties']['kunta']
    neighbourhood = feature['properties']['nimi']
    print(postal_code, municipality, neighbourhood)

00100 Helsinki Helsinki Keskusta - Etu-Töölö
00120 Helsinki Punavuori
00130 Helsinki Kaartinkaupunki
00140 Helsinki Kaivopuisto - Ullanlinna
00150 Helsinki Eira - Hernesaari
00160 Helsinki Katajanokka
00170 Helsinki Kruununhaka
00180 Helsinki Kamppi - Ruoholahti
00190 Helsinki Suomenlinna
00200 Helsinki Lauttasaari
00210 Helsinki Vattuniemi
00220 Helsinki Jätkäsaari
00230 Helsinki Ilmala
00240 Helsinki Länsi-Pasila
00250 Helsinki Taka-Töölö
00260 Helsinki Keski-Töölö
00270 Helsinki Pohjois-Meilahti
00280 Helsinki Ruskeasuo
00290 Helsinki Meilahden sairaala-alue
00300 Helsinki Pikku Huopalahti
00310 Helsinki Kivihaka
00320 Helsinki Etelä-Haaga
00330 Helsinki Munkkiniemi
00340 Helsinki Kuusisaari-Lehtisaari
00350 Helsinki Munkkivuori-Niemenmäki
00360 Helsinki Pajamäki
00370 Helsinki Reimarla
00380 Helsinki Pitäjänmäen teollisuusalue
00390 Helsinki Konala
00400 Helsinki Pohjois-Haaga
00440 Helsinki Lassila
00500 Helsinki Sörnäinen
00510 Helsinki Etu-Vallila - Alppila
00410 Helsinki Malmin

In [135]:
postal_codes_url = "https://fi.wikipedia.org/wiki/Luettelo_Suomen_postinumeroista_kunnittain"

In [2]:
#import required dependencies
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [2]:
#import required dependencies
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

In [109]:
address = 'Espoo, Uusimaa, Finland'

geolocator = Nominatim(user_agent="espoo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Espoo is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Espoo is 60.2241869, 24.6603626.


In [109]:
address = 'Espoo, Uusimaa, Finland'

geolocator = Nominatim(user_agent="espoo_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Espoo is {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Espoo is 60.2241869, 24.6603626.


In [110]:
# Generate at new folium map from latitude and longitude values
map_espoo = folium.Map(location=[latitude, longitude], zoom_start=11)

map_espoo

In [110]:
# Generate at new folium map from latitude and longitude values
map_espoo = folium.Map(location=[latitude, longitude], zoom_start=11)

map_espoo

In [3]:
# Set url
url = "https://fi.wikipedia.org/wiki/Luettelo_Suomen_postinumeroista_kunnittain"
# Query the page html
page = requests.get(url)
# Import the page into bs4
soup = BeautifulSoup(page.text, 'html.parser')

In [3]:
# Set url
url = "https://fi.wikipedia.org/wiki/Luettelo_Suomen_postinumeroista_kunnittain"
# Query the page html
page = requests.get(url)
# Import the page into bs4
soup = BeautifulSoup(page.text, 'html.parser')

In [114]:
#print(soup)
espoo_neighbourhoods = pd.DataFrame(columns=['PostalCode', 'Neighbourhood', 'Latitude', 'Longitude'])

postal_codes = soup.find_all('li')
for code in postal_codes:
    row = code.get_text()
    postal_code = re.findall(r'^[0-9]*', row)[0];
    if('Espoo / Esbo' in row and postal_code != ''):
        neighbourhoods = row.split('Espoo / Esbo –')[1].strip()
#        print(postal_code, neighbourhoods)
        
        address = postal_code + ', Espoo, Uusimaa, Finland'

        geolocator = Nominatim(user_agent="espoo_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
#        print(postal_code, latitude, longitude)
        espoo_neighbourhoods = espoo_neighbourhoods.append(
            { 'PostalCode': postal_code, 'Neighbourhood': neighbourhoods, 'Latitude': latitude, 'Longitude': longitude },
            ignore_index=True)


In [114]:
#print(soup)
espoo_neighbourhoods = pd.DataFrame(columns=['PostalCode', 'Neighbourhood', 'Latitude', 'Longitude'])

postal_codes = soup.find_all('li')
for code in postal_codes:
    row = code.get_text()
    postal_code = re.findall(r'^[0-9]*', row)[0];
    if('Espoo / Esbo' in row and postal_code != ''):
        neighbourhoods = row.split('Espoo / Esbo –')[1].strip()
#        print(postal_code, neighbourhoods)
        
        address = postal_code + ', Espoo, Uusimaa, Finland'

        geolocator = Nominatim(user_agent="espoo_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
#        print(postal_code, latitude, longitude)
        espoo_neighbourhoods = espoo_neighbourhoods.append(
            { 'PostalCode': postal_code, 'Neighbourhood': neighbourhoods, 'Latitude': latitude, 'Longitude': longitude },
            ignore_index=True)


In [115]:
espoo_neighbourhoods

,PostalCode,Neighbourhood,Latitude,Longitude
0,02100,"Tapiola – Heikintori, U-talo",60.177763,24.803189
1,02110,Tapiola – Itäranta,60.178228,24.816331
2,02120,"Tapiola – Suvikumpu, Hakalehto",60.172990,24.790880
3,02130,"Pohjois-Tapiola – Koivumankkaa, Pyhäristi",60.187425,24.793208
4,02140,"Laajalahti – Friisinmäki, Laajaranta",60.202770,24.805547
5,02150,"Otaniemi – Keilaniemi, Servinniemi, Teekkariky...",60.184174,24.829973
6,02160,Westend,60.167467,24.801453
7,02170,Haukilahti,60.159280,24.775274
8,02180,"Mankkaa – Seilimäki, Klovi, Taavinkylä",60.197960,24.761569
9,02200,"Henttaa, Niittykumpu – Piispankylä, Olarinluom...",60.172275,24.763034


In [115]:
espoo_neighbourhoods

,PostalCode,Neighbourhood,Latitude,Longitude
0,02100,"Tapiola – Heikintori, U-talo",60.177763,24.803189
1,02110,Tapiola – Itäranta,60.178228,24.816331
2,02120,"Tapiola – Suvikumpu, Hakalehto",60.172990,24.790880
3,02130,"Pohjois-Tapiola – Koivumankkaa, Pyhäristi",60.187425,24.793208
4,02140,"Laajalahti – Friisinmäki, Laajaranta",60.202770,24.805547
5,02150,"Otaniemi – Keilaniemi, Servinniemi, Teekkariky...",60.184174,24.829973
6,02160,Westend,60.167467,24.801453
7,02170,Haukilahti,60.159280,24.775274
8,02180,"Mankkaa – Seilimäki, Klovi, Taavinkylä",60.197960,24.761569
9,02200,"Henttaa, Niittykumpu – Piispankylä, Olarinluom...",60.172275,24.763034
